<a href="https://colab.research.google.com/github/DelmiroDaladier/Thermographic-Image-Analysis-for-Breast-Cancer-Classification-Using-Convolutional-Neural-Network/blob/main/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import statsmodels.tsa.api as smt 
import statistics
import numpy as np
import pandas.util.testing as tm



from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from scipy.stats import wasserstein_distance

from numpy.random import seed
from numpy import polyfit
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras import optimizers
from keras.models import Model
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Conv2D, Input, Flatten, concatenate, MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D, Dropout, BatchNormalization
from keras.regularizers import l2, l1, l1_l2
from keras.callbacks import EarlyStopping
from keras.metrics import Recall
from keras import initializers


In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

A célula anterior apenas faz um rescaling no valor dos pixels de cada imagem.

In [ ]:
def get_model():
    network_input = Input((250,250,3))

    convoluted = Conv2D(2,(5,5), activation='relu', kernel_regularizer=l2(0.01))(network_input)
    bn_1 = BatchNormalization()(convoluted)
    drop_1 = Dropout(0.2)(bn_1)

    convoluted_2 = Conv2D(4,(5,5), activation='relu')(drop_1)
    bn_2 = BatchNormalization()(convoluted_2)
    pool_2 = MaxPooling2D(pool_size=(3,3),strides=(1,1))(bn_2)

    convoluted_3 = Conv2D(8,(5,5), activation='relu', kernel_regularizer=l2(0.01))(pool_2)
    bn_3 = BatchNormalization()(convoluted_3)

    dense_2 = Dense(8, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01))(bn_3)
    bn_6 = BatchNormalization()(dense_2)
    drop_6 = Dropout(0.5)(bn_6)

    flattened = Flatten()(drop_6)
    output = Dense(3, activation='softmax')(flattened)
    model = Model(network_input,output)

    return model

A célula seguinte plota um modelo gráfico da rede que foi criada na célula anterior. Lembrando que para executar o experimento com duas classes devemos modificar a camada Dense para Dense(3, activation='sigmoid')(flattened)

In [ ]:
model = get_model()
#plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
optimizer = optimizers.Adam(lr=0.00001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy',
             'categorical_accuracy',
             Recall(class_id=1)
             ]
)

**Experimento utilizando 3 classes**

In [ ]:
import tensorflow as tf

In [ ]:
np.random.seed(42)
for fold in range(0, 10):
    train_directory_path = f'drive/My Drive/Colab Notebooks/dataset_6/dataset_6/fold_{fold}/train'
    test_directory_path =  f'drive/My Drive/Colab Notebooks/dataset_6/dataset_6/fold_{fold}/test'
    
   
    print(f'Fold: {fold}\n\n')
    # loss='binary_crossentropy',
    a = model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy',
             'categorical_accuracy',
             Recall(class_id=1),
             ]
    )


    seed = 42
    batch_size = 1
    train_data = datagen.flow_from_directory(
                                            train_directory_path,
                                            target_size = (250,250),                                      
                                            class_mode = 'categorical',
                                            color_mode='rgb', 
                                            batch_size=batch_size,
                                            shuffle=True,
                                            # seed=seed
                                            )

    test_data = validation_datagen.flow_from_directory(
                                            test_directory_path,
                                            target_size = (250,250),
                                            class_mode = 'categorical',
                                            color_mode='rgb',
                                            batch_size=batch_size,
                                            shuffle=True,
                                            # seed=seed
                                          )
    my_callbacks = [
        EarlyStopping(
            monitor='val_categorical_accuracy',
            min_delta=0,
            patience=2,
            verbose=1,
            mode='auto'
            )
    ]

    np.random.seed(42)
    fit_model = model.fit(
                                    train_data,
                                    steps_per_epoch = 324,
                                    epochs=150,
                                    validation_data=test_data,
                                    validation_steps=36,
                                    use_multiprocessing=False
    )
    


Fold: 0


Found 486 images belonging to 3 classes.
Found 54 images belonging to 3 classes.
Epoch 1/150
324/324 [==============================] - 3s 10ms/step - loss: 1.3066 - accuracy: 0.4475 - categorical_accuracy: 0.4475 - recall_28: 0.3814 - val_loss: 1.1808 - val_accuracy: 0.2778 - val_categorical_accuracy: 0.2778 - val_recall_28: 0.0000e+00
Epoch 2/150
324/324 [==============================] - 3s 8ms/step - loss: 0.9038 - accuracy: 0.6420 - categorical_accuracy: 0.6420 - recall_28: 0.6147 - val_loss: 1.0216 - val_accuracy: 0.5278 - val_categorical_accuracy: 0.5278 - val_recall_28: 0.0833
Epoch 3/150
324/324 [==============================] - 3s 8ms/step - loss: 0.6588 - accuracy: 0.7593 - categorical_accuracy: 0.7593 - recall_28: 0.7387 - val_loss: 1.2517 - val_accuracy: 0.5278 - val_categorical_accuracy: 0.5278 - val_recall_28: 0.1000
Epoch 4/150
324/324 [==============================] - 3s 8ms/step - loss: 0.5205 - accuracy: 0.8457 - categorical_accuracy: 0.8457 - recall_28: 

**Experimento Utilizando apenas duas classes**

In [ ]:
def get_model():
    network_input = Input((250,250,3))

    convoluted = Conv2D(2,(5,5), activation='relu', kernel_regularizer=l2(0.01))(network_input)
    bn_1 = BatchNormalization()(convoluted)
    drop_1 = Dropout(0.2)(bn_1)

    convoluted_2 = Conv2D(4,(5,5), activation='relu')(drop_1)
    bn_2 = BatchNormalization()(convoluted_2)
    pool_2 = MaxPooling2D(pool_size=(3,3),strides=(1,1))(bn_2)

    convoluted_3 = Conv2D(8,(5,5), activation='relu', kernel_regularizer=l2(0.01))(pool_2)
    bn_3 = BatchNormalization()(convoluted_3)

    dense_2 = Dense(8, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01))(bn_3)
    bn_6 = BatchNormalization()(dense_2)
    drop_6 = Dropout(0.5)(bn_6)

    flattened = Flatten()(drop_6)
    output = Dense(1, activation='sigmoid')(flattened)
    model = Model(network_input,output)

    return model

In [ ]:
model = get_model()
#plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
optimizer = optimizers.Adam(lr=0.00001)

In [ ]:
np.random.seed(42)
for fold in range(0, 10):
    train_directory_path = f'drive/My Drive/Colab Notebooks/dataset_7/dataset_7/fold_{fold}/train'
    test_directory_path =  f'drive/My Drive/Colab Notebooks/dataset_7/dataset_7/fold_{fold}/test'
    
    print(f'Fold: {fold}\n\n')
    #loss='categorical_crossentropy',
    model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy',
             'binary_accuracy',
             Recall(class_id=0)
             ]
    )


    seed = 42
    batch_size = 1
    train_data = datagen.flow_from_directory(
                                            train_directory_path,
                                            target_size = (250,250),
                                            class_mode = 'binary',
                                            color_mode='rgb', 
                                            batch_size=batch_size,
                                            shuffle=True 
                                            )

    test_data = validation_datagen.flow_from_directory(
                                            test_directory_path,
                                            target_size = (250,250),
                                            class_mode = 'binary',
                                            color_mode='rgb',
                                            batch_size=batch_size,
                                            shuffle=True 
                                          )
    np.random.seed(42)
    fit_model = model.fit           (
                                    train_data,
                                    steps_per_epoch = 324,
                                    epochs=150,
                                    validation_data=test_data,
                                    validation_steps=36,
                                    use_multiprocessing=False,
                                   
    )



Fold: 0


Found 324 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Epoch 1/150
324/324 [==============================] - 3s 10ms/step - loss: 0.8160 - accuracy: 0.5093 - binary_accuracy: 0.5093 - recall_38: 0.4630 - val_loss: 0.7681 - val_accuracy: 0.5000 - val_binary_accuracy: 0.5000 - val_recall_38: 1.0000
Epoch 2/150
324/324 [==============================] - 3s 8ms/step - loss: 0.6947 - accuracy: 0.6574 - binary_accuracy: 0.6574 - recall_38: 0.6235 - val_loss: 0.7313 - val_accuracy: 0.6389 - val_binary_accuracy: 0.6389 - val_recall_38: 1.0000
Epoch 3/150
324/324 [==============================] - 3s 8ms/step - loss: 0.6099 - accuracy: 0.7222 - binary_accuracy: 0.7222 - recall_38: 0.6420 - val_loss: 0.6526 - val_accuracy: 0.6389 - val_binary_accuracy: 0.6389 - val_recall_38: 1.0000
Epoch 4/150
324/324 [==============================] - 3s 8ms/step - loss: 0.5364 - accuracy: 0.7593 - binary_accuracy: 0.7593 - recall_38: 0.7654 - val_loss: 0.6215 - val_accurac

In [ ]:
np.random.seed(42)
for fold in range(0, 10):
    train_directory_path = f'drive/My Drive/Colab Notebooks/dataset_7/dataset_7/fold_{fold}/train'
    test_directory_path =  f'drive/My Drive/Colab Notebooks/dataset_7/dataset_7/fold_{fold}/test'
    
    print(f'Fold: {fold}\n\n')
    #loss='categorical_crossentropy',
    model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy',
             'binary_accuracy',
             Recall(class_id=0)
             ]
    )


    seed = 42
    batch_size = 1
    train_data = datagen.flow_from_directory(
                                            train_directory_path,
                                            target_size = (250,250),
                                            class_mode = 'binary',
                                            color_mode='rgb', 
                                            batch_size=batch_size,
                                            shuffle=True 
                                            )

    test_data = validation_datagen.flow_from_directory(
                                            test_directory_path,
                                            target_size = (250,250),
                                            class_mode = 'binary',
                                            color_mode='rgb',
                                            batch_size=batch_size,
                                            shuffle=True 
                                          )
    my_callbacks = [
        EarlyStopping(
            min_delta=0,
            patience=2,
            verbose=1,
            mode='auto'
            )
    ]
    
    np.random.seed(42)
    fit_model = model.fit           (
                                    train_data,
                                    steps_per_epoch = 324,
                                    epochs=150,
                                    validation_data=test_data,
                                    validation_steps=36,
                                    use_multiprocessing=False,
                                   
    )



Fold: 0


Found 324 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Epoch 1/150
324/324 [==============================] - 3s 10ms/step - loss: 0.8236 - accuracy: 0.5525 - binary_accuracy: 0.5525 - recall_48: 0.5741 - val_loss: 0.7690 - val_accuracy: 0.5556 - val_binary_accuracy: 0.5556 - val_recall_48: 0.5000
Epoch 2/150
324/324 [==============================] - 3s 8ms/step - loss: 0.6112 - accuracy: 0.7407 - binary_accuracy: 0.7407 - recall_48: 0.7716 - val_loss: 0.7571 - val_accuracy: 0.6389 - val_binary_accuracy: 0.6389 - val_recall_48: 0.7778
Epoch 3/150
324/324 [==============================] - 3s 8ms/step - loss: 0.4394 - accuracy: 0.8735 - binary_accuracy: 0.8735 - recall_48: 0.8951 - val_loss: 0.7939 - val_accuracy: 0.6944 - val_binary_accuracy: 0.6944 - val_recall_48: 0.9444
Epoch 4/150
324/324 [==============================] - 3s 9ms/step - loss: 0.3568 - accuracy: 0.9321 - binary_accuracy: 0.9321 - recall_48: 0.9444 - val_loss: 0.5695 - val_accurac